# **1. 서울 열린 데이터 광장**
서울 열린데이터 광장(Seoul Open Data Plaza)은 서울시에서 운영하는 공공데이터 개방 플랫폼입니다. 시민, 연구자, 기업 등이 서울시에서 생성한 다양한 공공데이터를 자유롭게 활용할 수 있도록 제공하고 있습니다. 이를 통해 데이터 기반의 창의적인 아이디어와 혁신을 촉진하며, 시민들의 정보 접근성을 높이고 공공서비스를 개선하는 데 기여하고 있습니다.
* [공식 사이트](https://data.seoul.go.kr/)

In [1]:
import requests
import folium
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
base_url='http://openapi.seoul.go.kr:8088/686f6e56717465723738506a41586d/json/bikeList/1/3/'
response = requests.get(base_url)
# print(response)
json_data = response.json()
json_data

{'rentBikeStatus': {'list_total_count': 3,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
  'row': [{'rackTotCnt': '15',
    'stationName': '102. 망원역 1번출구 앞',
    'parkingBikeTotCnt': '21',
    'shared': '140',
    'stationLatitude': '37.55564880',
    'stationLongitude': '126.91062927',
    'stationId': 'ST-4'},
   {'rackTotCnt': '14',
    'stationName': '103. 망원역 2번출구 앞',
    'parkingBikeTotCnt': '20',
    'shared': '143',
    'stationLatitude': '37.55495071',
    'stationLongitude': '126.91083527',
    'stationId': 'ST-5'},
   {'rackTotCnt': '13',
    'stationName': '104. 합정역 1번출구 앞',
    'parkingBikeTotCnt': '0',
    'shared': '0',
    'stationLatitude': '37.55073929',
    'stationLongitude': '126.91508484',
    'stationId': 'ST-6'}]}}

In [5]:
json_data.get('rentBikeStatus', {}).get('RESULT', {}).get('CODE', '')

'INFO-000'

In [6]:
json_data['rentBikeStatus']['RESULT']['CODE']

'INFO-000'

In [10]:
def fetch_bike_data():
    base_url = "http://openapi.seoul.go.kr:8088/686f6e56717465723738506a41586d/json/bikeList/"
    start = 1
    end = 1000
    step = 1000
    data_frames = []

    while True:
        url = f"{base_url}{start}/{end}/"
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Status Code: {response.status_code}")
            break

        json_data = response.json()

        try:
            rent_bike_status = json_data["rentBikeStatus"]
            result_code = rent_bike_status["RESULT"]["CODE"]
        except KeyError:
            print("JSON 오류")
            break

        if result_code == "INFO-200":
            print("데이터 없음")
            break
        elif result_code == "INFO-000":
            print(f"시작: {start} 끝: {end}.")
            try:
                bike_data = rent_bike_status["row"]
                if bike_data:
                    df = pd.DataFrame(bike_data)
                    data_frames.append(df)
            except KeyError:
                print("데이터를 찾을 수 없음")
        else:
            print(f"result code: {result_code}")
            break

        start += step
        end += step

    if data_frames:
        final_df = pd.concat(data_frames, ignore_index=True)
        return final_df
    else:
        return pd.DataFrame()

In [12]:
bike_data_df = fetch_bike_data()
bike_data_df

시작: 1 끝: 1000.
시작: 1001 끝: 2000.
시작: 2001 끝: 3000.
JSON 오류


,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId
0,15,102. 망원역 1번출구 앞,19,127,37.55564880,126.91062927,ST-4
1,14,103. 망원역 2번출구 앞,22,157,37.55495071,126.91083527,ST-5
2,13,104. 합정역 1번출구 앞,0,0,37.55073929,126.91508484,ST-6
3,5,105. 합정역 5번출구 앞,5,100,37.55000687,126.91482544,ST-7
4,12,106. 합정역 7번출구 앞,13,108,37.54864502,126.91282654,ST-8
...,...,...,...,...,...,...,...
2709,10,6172. 가양5단지아파트,0,0,37.56447983,126.85464478,ST-3281
2710,10,6173. 서울자동차운전전문학원,4,40,37.54613495,126.82106018,ST-3284
2711,7,6176. 유광사 여성병원 앞,2,29,37.54344940,126.83891296,ST-3327
2712,10,6177. 마곡롯데캐슬르웨스트,0,0,37.56356049,126.82462311,ST-3324


In [13]:
bike_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2714 entries, 0 to 2713
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   rackTotCnt         2714 non-null   object
 1   stationName        2714 non-null   object
 2   parkingBikeTotCnt  2714 non-null   object
 3   shared             2714 non-null   object
 4   stationLatitude    2714 non-null   object
 5   stationLongitude   2714 non-null   object
 6   stationId          2714 non-null   object
dtypes: object(7)
memory usage: 148.5+ KB


In [14]:
'''
rackTotCnt	거치대개수
parkingBikeTotCnt	자전거주차총건수
shared	거치율
stationLatitude	위도
stationLongitude	경도
stationId	대여소ID
stationName	대여소이름
'''
bike_data_df.columns

Index(['rackTotCnt', 'stationName', 'parkingBikeTotCnt', 'shared',
       'stationLatitude', 'stationLongitude', 'stationId'],
      dtype='object')

In [15]:
bike_data_df['stationLongitude'] = bike_data_df['stationLongitude'].astype(float)
bike_data_df['stationLatitude'] = bike_data_df['stationLatitude'].astype(float)

In [16]:
bike_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2714 entries, 0 to 2713
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rackTotCnt         2714 non-null   object 
 1   stationName        2714 non-null   object 
 2   parkingBikeTotCnt  2714 non-null   object 
 3   shared             2714 non-null   object 
 4   stationLatitude    2714 non-null   float64
 5   stationLongitude   2714 non-null   float64
 6   stationId          2714 non-null   object 
dtypes: float64(2), object(5)
memory usage: 148.5+ KB


In [17]:
bike_map = folium.Map(location=[bike_data_df['stationLatitude'].mean(),
                                bike_data_df['stationLongitude'].mean()],
                                zoom_start=12)

for index, data in bike_data_df.iterrows():
    popup_str = '{} 자전거주차총건수:{}대'.format(
        data['stationName'], data['parkingBikeTotCnt']
    )
    popup = folium.Popup(popup_str, max_width=600)
    folium.Marker(location=[data['stationLatitude'], data['stationLongitude']],
                  popup=popup).add_to(bike_map)

bike_map

Output hidden; open in https://colab.research.google.com to view.